In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:

!git clone https://github.com/haffi96/eng-feed # replace any repository of your choice

Cloning into 'eng-feed'...
remote: Enumerating objects: 864, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 864 (delta 52), reused 103 (delta 41), pack-reused 727
Receiving objects: 100% (864/864), 1.60 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (455/455), done.


In [71]:
from langchain.embeddings.ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mistral")

In [72]:
import pathlib
import glob

path = pathlib.Path.home() / "code" / "eng-feed"

all_files = []

ts_files = all_files.extend(glob.glob("./eng-feed" + '/**/*.ts', recursive=True))
astro_files = all_files.extend(glob.glob("./eng-feed" + '/**/*.astro', recursive=True))
md_files = all_files.extend(glob.glob("./eng-feed" + '/**/*.md', recursive=True))
json_files = all_files.extend(glob.glob("./eng-feed" + '/**/*.json', recursive=True))
js_files = all_files.extend(glob.glob("./eng-feed" + '/**/*.js', recursive=True))


In [73]:
import os
from langchain.document_loaders import TextLoader

# root_dir = "./eng-feed/src"
docs = []
for file in all_files:
    try:
        loader = TextLoader(file_path=file, encoding="utf-8")
        docs.extend(loader.load_and_split())
    except Exception as e:
        pass
    

In [75]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)


In [77]:
from langchain.vectorstores import DeepLake

deeplake_username = "haffimazhar96"

db = DeepLake(
    dataset_path=f"hub://{deeplake_username}/test",
    embedding=embeddings,
)
db.add_documents(texts)

Your Deep Lake dataset has been successfully created!


In [29]:
db = DeepLake(
    dataset_path=f"hub://{deeplake_username}/test",
    read_only=True,
    embedding=embeddings,
)

Deep Lake Dataset in hub://haffimazhar96/eng-feed already exists, loading from the storage


In [30]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 10

In [31]:
from langchain.chat_models import ChatOllama
from langchain.chains import ConversationalRetrievalChain

model = ChatOllama(model="mistral")
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [32]:
questions = [
    "What does this code repo do?",
    "Where is this app hosted?",
    "How do users get notified when there is a new post?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What does this code repo do? 

**Answer**: 
This code snippet is a shell script that implements a pre-rebase hook for Git. The script checks whether the branch being rebased (or empty if rebasing the current branch) is already merged to the publish branch ("next" in this example), and takes different actions based on that:

1. If the topic branch has ever been merged to "next", it prevents the rebase from happening by exiting with a non-zero status. This ensures that previously published history is not overwritten.
2. If the topic branch has been fully merged to "master", it can be safely deleted.
3. If the topic branch still exists and needs to be rebased on top of the current branch, the script does the rebase while excluding the "publish" branch (which contains already published history).

The script also checks whether the repository is in a detached HEAD state or not. If it's not, it skips the pre-rebase hook step, indicating that the repository isn't ready for a 

In [ ]:
new_question = [
    ""
]

for question in new_question:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")